## NeuroDet: CNN-LSTM Model

### Import Libraries

In [3]:
import os
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds
import PIL
import PIL.Image
import matplotlib.pyplot as plt
import pathlib
import math

# Disabling GPU for the moment because of the lack of the memory
os.environ["CUDA_VISIBLE_DEVICES"] = "-1" 
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  0


### Loading the dataset of Brain Scan images
#### Full data for training and testing
Source of the Dataset: [Kaggle-Brain Tumor Classification](https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri?select=Testing)<br>
Reference for operations performed : [Tensorflow tutorial: Load Images](https://www.tensorflow.org/tutorials/load_data/images)

In [4]:
categories_path = {'glioma_tumor': '/glioma_tumor', 'meningioma_tumor': '/meningioma_tumor', 
                   'pituitary_tumor': '/pituitary_tumor', 'no_tumor':'/no_tumor'}
train_path = 'BrainMRI/Training'
test_path = 'BrainMRI/Testing'

# train_glioma_dir = pathlib.Path(train_path + categories_path['glioma_tumor'])
train_dir = pathlib.Path(train_path)
test_dir = pathlib.Path(test_path)

# Training data: number of examples
# label = 0 | glioma_tumor: 826
# label = 1 | meningioma_tumor: 822
# label = 2 | no_tumor: 395 
# label = 3 | pituitary_tumor: 827 

# Testing data: number of examples
# label = 0 | glioma_tumor: 100
# label = 1 | meningioma_tumor: 115
# label = 2 | no_tumor: 105
# label = 3 | pituitary_tumor: 74

num_train_examples_dict = {
    "label_0": 826, "label_1": 822, "label_2": 395, "label_3": 827}

num_test_examples_dict = {
    "label_0": 100, "label_1": 115, "label_2": 105, "label_3": 74}


num_train_examples = sum(num_train_examples_dict.values()) # 2870
num_test_examples = sum(num_test_examples_dict.values()) # 394

print(f'Total number of train examples: {num_train_examples}')
print(f'Total number of test examples: {num_test_examples}') 

# Defining the parameters of the dataset
batch_size = 128
img_height = 256
img_width = 256 

Total number of train examples: 2870
Total number of test examples: 394


#### 4-Class Classification Data 

In [6]:
# Loading the train dataset using keras.utils.image_dataset_from_directory
# To use this method, please ensure you have tf.nigthly installed 
train_data_full = tf.keras.utils.image_dataset_from_directory(
    train_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_train_examples, 
    shuffle=True,
    color_mode = 'grayscale')

test_data_full = tf.keras.utils.image_dataset_from_directory(
    test_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_test_examples, 
    shuffle=True,
    color_mode = 'grayscale')

print(train_data_full.class_names)
print(test_data_full.class_names)

Found 2870 files belonging to 4 classes.
Found 394 files belonging to 4 classes.
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']
['glioma_tumor', 'meningioma_tumor', 'no_tumor', 'pituitary_tumor']


#### 2-Class Classification Data

In [7]:
train2_path = 'BrainMRI_2Class/Training'
test2_path = 'BrainMRI_2Class/Testing'

train2_dir = pathlib.Path(train2_path)
test2_dir = pathlib.Path(test2_path)

# Loading the train dataset using keras.utils.image_dataset_from_directory
# To use this method, please ensure you have tf.nigthly installed 
train2_data_full = tf.keras.utils.image_dataset_from_directory(
    train2_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_train_examples, 
    color_mode = 'grayscale')

test2_data_full = tf.keras.utils.image_dataset_from_directory(
    test2_dir, seed=123,
    image_size=(img_height, img_width),
    batch_size=num_test_examples, 
    color_mode = 'grayscale')

print(train2_data_full.class_names)
print(test2_data_full.class_names)

Found 2870 files belonging to 2 classes.
Found 394 files belonging to 2 classes.
['no_tumor', 'tumor']
['no_tumor', 'tumor']


### Normalizing the training and testing set

In [8]:
normalization_layer = tf.keras.layers.Rescaling(1./255)

train_data_full = train_data_full.map(lambda x, y: (normalization_layer(x), y))
test_data_full = test_data_full.map(lambda x, y: (normalization_layer(x), y))

train2_data_full = train2_data_full.map(lambda x, y: (normalization_layer(x), y))
test2_data_full = test2_data_full.map(lambda x, y: (normalization_layer(x), y))

### Separate data into images and labels for each set

In [9]:
for images, labels in train_data_full:
    x_train = images
    y_train = labels
    
for images, labels in test_data_full:
    x_test = images
    y_test = labels
    
for images, labels in train2_data_full:
    x_train2 = images
    y_train2 = labels
    
for images, labels in test2_data_full:
    x_test2 = images
    y_test2 = labels

# 4-Class CNN-LSTM Model

In [54]:
size_lstm = 128
num_epochs = 50
learning_rate = 0.005
dense1_size = 80
dense2_size = 4
filter1_size = 50
filter2_size = 20
filter3_size = 1
kernel_size = 3
dropout_rate = 0.2


model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(batch_input_shape=(None,img_height,img_width,1)))

# CNN
# Convolutional layer 1, batch normalization, max pooling
model.add(tf.keras.layers.Conv2D(
    filters=filter1_size, 
    kernel_size=(kernel_size,kernel_size), 
    padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(padding="same"))

# Convolutional layer 2, batch normalization, max pooling
model.add(tf.keras.layers.Conv2D(
    filters=filter2_size, 
    kernel_size=(kernel_size,kernel_size), 
    padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(padding="same"))

# Convolutional layer 3, batch normalization, max pooling
model.add(tf.keras.layers.Conv2D(
    filters=filter3_size, 
    kernel_size=(kernel_size,kernel_size), 
    padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(padding="same"))

# LSTM
model.add(tf.keras.layers.Reshape((32,32)))
model.add(tf.keras.layers.LSTM(units=size_lstm))
model.add(tf.keras.layers.Dropout(dropout_rate))
model.add(tf.keras.layers.Dense(dense1_size, activation='relu'))
model.add(tf.keras.layers.Dropout(dropout_rate))
model.add(tf.keras.layers.Dense(dense2_size, activation='softmax'))

# Train model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['acc'])

model.fit(x_train, y_train, epochs=num_epochs)
model.summary()

# Evaluate model
test_loss, test_acc = model.evaluate(x_test, y_test)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_acc}')   


x_train.shape = (2870, 256, 256, 1)
Epoch 1/50
90/90 [==============================] - 102s 1s/step - loss: 1.0746 - acc: 0.5380
Epoch 2/50
90/90 [==============================] - 100s 1s/step - loss: 0.7799 - acc: 0.6735
Epoch 3/50
90/90 [==============================] - 104s 1s/step - loss: 0.6984 - acc: 0.7084
Epoch 4/50
90/90 [==============================] - 102s 1s/step - loss: 0.5937 - acc: 0.7481
Epoch 5/50
90/90 [==============================] - 100s 1s/step - loss: 0.5266 - acc: 0.7829
Epoch 6/50
90/90 [==============================] - 101s 1s/step - loss: 0.5117 - acc: 0.7878
Epoch 7/50
90/90 [==============================] - 104s 1s/step - loss: 0.4510 - acc: 0.8195
Epoch 8/50
90/90 [==============================] - 102s 1s/step - loss: 0.3794 - acc: 0.8411
Epoch 9/50
90/90 [==============================] - 102s 1s/step - loss: 0.2994 - acc: 0.8774
Epoch 10/50
90/90 [==============================] - 103s 1s/step - loss: 0.2789 - acc: 0.8836
Epoch 11/50
90/90 [====

# 2 class CNN-LSTM

In [20]:
size_lstm = 128
num_epochs = 50
learning_rate = 0.005
dense1_size = 80
dense2_size = 2
filter1_size = 50
filter2_size = 20
filter3_size = 1
kernel_size = 3
dropout_rate = 0.2

model = tf.keras.Sequential()
model.add(tf.keras.layers.Input(batch_input_shape=(None,img_height,img_width,1)))

# CNN
# Convolutional layer 1, batch normalization, max pooling
model.add(tf.keras.layers.Conv2D(
    filters=filter1_size, 
    kernel_size=(kernel_size,kernel_size), 
    padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(padding="same"))

# Convolutional layer 2, batch normalization, max pooling
model.add(tf.keras.layers.Conv2D(
    filters=filter2_size, 
    kernel_size=(kernel_size,kernel_size), 
    padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(padding="same"))

# Convolutional layer 3, batch normalization, max pooling
model.add(tf.keras.layers.Conv2D(
    filters=filter3_size, 
    kernel_size=(kernel_size,kernel_size), 
    padding="same"))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(padding="same"))

# LSTM
model.add(tf.keras.layers.Reshape((32,32)))
model.add(tf.keras.layers.LSTM(units=size_lstm))
model.add(tf.keras.layers.Dropout(dropout_rate ))
model.add(tf.keras.layers.Dense(dense1_size, activation='relu'))
model.add(tf.keras.layers.Dropout(dropout_rate ))
model.add(tf.keras.layers.Dense(dense2_size, activation='softmax'))

# Train model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
    loss='sparse_categorical_crossentropy',
    metrics=['acc'])

model.fit(x_train2, y_train2, epochs=num_epochs)
model.summary()

# Evaluate model
test_loss, test_acc = model.evaluate(x_test2, y_test2)

print(f'Testing loss = {test_loss}, Testing accuracy = {test_acc}')   



Epoch 1/50
90/90 [==============================] - 128s 1s/step - loss: 0.3620 - acc: 0.8523
Epoch 2/50
90/90 [==============================] - 127s 1s/step - loss: 0.3029 - acc: 0.8787
Epoch 3/50
90/90 [==============================] - 126s 1s/step - loss: 0.2552 - acc: 0.9000
Epoch 4/50
90/90 [==============================] - 123s 1s/step - loss: 0.2515 - acc: 0.8944
Epoch 5/50
90/90 [==============================] - 123s 1s/step - loss: 0.2322 - acc: 0.9063
Epoch 6/50
90/90 [==============================] - 120s 1s/step - loss: 0.2448 - acc: 0.8990
Epoch 7/50
90/90 [==============================] - 117s 1s/step - loss: 0.1858 - acc: 0.9178
Epoch 8/50
90/90 [==============================] - 97s 1s/step - loss: 0.1661 - acc: 0.9348
Epoch 9/50
90/90 [==============================] - 99s 1s/step - loss: 0.1624 - acc: 0.9352
Epoch 10/50
90/90 [==============================] - 124s 1s/step - loss: 0.1372 - acc: 0.9446
Epoch 11/50
90/90 [==============================] - 127s 1s/